In [34]:
%load_ext autoreload
%autoreload 2

from project.data.data import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess raw data:

In [2]:
processor = RaceDataProcessor()
processor.process_data("RACE", "LON")

27933it [01:50, 252.43it/s]

Preprocess data successfully!


## Prepare datasets:

##### All of the following steps should be done from command line. They are here to a quick & dirty heuristic to work with jupyter.

In [2]:
from collections import namedtuple

Hparams = namedtuple("Hparams", ["data_path", "batch_size", "num_workers", "pretrained_model"])
hparams = Hparams(
    data_path = "LON",
    batch_size = 8,
    num_workers = 6,
    pretrained_model = "distilbert-base-cased"
)

In [3]:
# Create and setup 
lit_data_module = RaceDataModule(hparams)
lit_data_module.prepare_data()
lit_data_module.setup()

In [4]:
# Get data loaders:
trainloader = lit_data_module.train_dataloader()
valloader = lit_data_module.val_dataloader()
testloader = lit_data_module.test_dataloader()

## Create model:

In [8]:
import torch
from transformers import AutoModel
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [18]:
device = torch.device("cpu")

In [5]:
cac = iter(trainloader)

In [6]:
batch = next(cac)

In [19]:
for k, v in batch["articles"].items():
    batch["articles"][k] = v.to(device)
for k, v in batch["questions"].items():
    batch["questions"][k] = v.to(device)

In [35]:
from project.models.models import *

In [36]:
Hparams = namedtuple("Hparams", ["d_model", "nhead", "num_layers", "learning_rate", "pretrained_model"])
hparams = Hparams(
    d_model = 768,
    nhead = 8,
    num_layers = 1,
    learning_rate = 1e-3,
    pretrained_model = "distilbert-base-cased"
)

race_model = RaceModule(hparams).to(device)

In [37]:
output, encode, decode, tgt = race_model(batch["questions"], batch["questions"])

In [38]:
output

tensor([[[6.4301e-05, 1.2961e-05, 1.9590e-05,  ..., 2.8862e-05,
          3.4240e-05, 6.0571e-05],
         [6.2088e-05, 1.2162e-05, 2.0187e-05,  ..., 4.2339e-05,
          3.6512e-05, 5.0738e-05],
         [5.7301e-05, 1.2832e-05, 1.9295e-05,  ..., 3.8062e-05,
          3.0018e-05, 4.0314e-05],
         ...,
         [7.8482e-05, 1.3128e-05, 2.1414e-05,  ..., 4.0758e-05,
          3.4472e-05, 4.6963e-05],
         [5.2535e-05, 1.1564e-05, 1.8495e-05,  ..., 4.1640e-05,
          3.2433e-05, 3.8565e-05],
         [6.1419e-05, 1.4153e-05, 1.8233e-05,  ..., 3.7667e-05,
          3.3458e-05, 4.7542e-05]],

        [[6.1181e-05, 9.5091e-06, 2.8553e-05,  ..., 3.4644e-05,
          1.7745e-05, 2.7025e-05],
         [8.7221e-05, 1.8367e-05, 1.5286e-05,  ..., 3.5893e-05,
          3.0031e-05, 3.0061e-05],
         [7.8879e-05, 1.8169e-05, 3.9975e-05,  ..., 1.8118e-05,
          5.8365e-05, 4.4215e-05],
         ...,
         [7.2393e-05, 1.5365e-05, 1.9110e-05,  ..., 3.5657e-05,
          3.304